In [2]:
%load_ext autoreload
%autoreload 2
import os
import re
import json
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from pydub import AudioSegment
from scipy.io.wavfile import read, write
from pipefunc.pipefunc import do_command, send_command, get_response, get_response_json

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
pipe-test.py, running on windows
Write to  "\\.\pipe\ToSrvPipe"
Read from "\\.\pipe\FromSrvPipe"
-- Both pipes exist.  Good.


OSError: [Errno 22] Invalid argument: '\\\\.\\pipe\\ToSrvPipe'

In [38]:
audio_prefix = 'EP20'
namelistorder = ['HWC', 'Angel', 'YY']

p = Path(os.getcwd())

p_raw = p.parents[0].joinpath('postedit')
p_mono = p.parents[0].joinpath('postedit_mono')
p_mono.mkdir(exist_ok=True)
p_scripts = p.parents[0].joinpath('scripts')

folderlist = [x.name for x in sorted(p_raw.iterdir())]

print(folderlist)
ipfilelist_dict = {}
opmonofilelist = {}

for name in namelistorder:
    ipfilelist_dict[name] = {}
    for fldname in folderlist:
        ipfilelist_dict[name][fldname] = sorted(p_raw.joinpath(fldname).glob(f'*{name}*.wav'))
    opmonofilelist[name] = p_mono.joinpath(f'{audio_prefix}_{name}.wav')

print(ipfilelist_dict)
print(opmonofilelist)

['Section 1', 'Section 2', 'Section 3', 'Section 4']
{'HWC': {'Section 1': [WindowsPath('c:/Users/michaelshih/Documents/Podcast_tmp/EP020/postedit/Section 1/Ep20_HWC_section1.wav')], 'Section 2': [WindowsPath('c:/Users/michaelshih/Documents/Podcast_tmp/EP020/postedit/Section 2/Ep20_HWC_section2.wav')], 'Section 3': [WindowsPath('c:/Users/michaelshih/Documents/Podcast_tmp/EP020/postedit/Section 3/Ep20_HWC_section3.wav')], 'Section 4': [WindowsPath('c:/Users/michaelshih/Documents/Podcast_tmp/EP020/postedit/Section 4/Ep20_HWC_section4.wav')]}, 'Angel': {'Section 1': [WindowsPath('c:/Users/michaelshih/Documents/Podcast_tmp/EP020/postedit/Section 1/Ep20_Angel_section1.wav')], 'Section 2': [WindowsPath('c:/Users/michaelshih/Documents/Podcast_tmp/EP020/postedit/Section 2/Ep20_Angel_section2.wav')], 'Section 3': [WindowsPath('c:/Users/michaelshih/Documents/Podcast_tmp/EP020/postedit/Section 3/Ep20_Angel_section3.wav')], 'Section 4': [WindowsPath('c:/Users/michaelshih/Documents/Podcast_tmp/EP02

In [39]:
def createmetadata(ipfilelist_dict, folderlist, namelistorder, jsonname = 'metadata.json'):
    if p_scripts.joinpath(jsonname).is_file():
            with open(p_scripts.joinpath(jsonname), 'r') as f:
                mainmetadata_dict = json.load(f)
    else: 
        metadata_dict = {}
        for section in folderlist:
            metadata_dict[section] = {}
            wavfilelist = [ipfilelist_dict[x][section][0] for x in namelistorder]
            
            for wavfile in wavfilelist:
                fs, data = read(wavfile)
                metadata_dict[section][wavfile.name] = {'fs': fs, 
                                                'shape': data.shape,
                                                }
        mainmetadata_dict = {}
        mainmetadata_dict['postedit'] = metadata_dict
        with open(p_scripts.joinpath(jsonname), 'w') as f:
            json.dump(mainmetadata_dict, f)
    
    return mainmetadata_dict

mainmetadata_dict = createmetadata(ipfilelist_dict, folderlist, namelistorder, jsonname = 'metadata.json')

c:\Users\michaelshih\Documents\Podcast_tmp\EP020\postedit\Section 1\Ep20_HWC_section1.wav
44100
(17518491,)
c:\Users\michaelshih\Documents\Podcast_tmp\EP020\postedit\Section 1\Ep20_Angel_section1.wav
44100
(17518491,)
c:\Users\michaelshih\Documents\Podcast_tmp\EP020\postedit\Section 1\Ep20_YY_section1.wav
44100
(17518491,)
c:\Users\michaelshih\Documents\Podcast_tmp\EP020\postedit\Section 2\Ep20_HWC_section2.wav
44100
(108275654,)
c:\Users\michaelshih\Documents\Podcast_tmp\EP020\postedit\Section 2\Ep20_Angel_section2.wav


C:\Users\michaelshih\AppData\Local\Temp\ipykernel_66340\2939306947.py:13: WavFileWarning: Chunk (non-data) not understood, skipping it.
  fs, data = read(wavfile)


44100
(108275654,)
c:\Users\michaelshih\Documents\Podcast_tmp\EP020\postedit\Section 2\Ep20_YY_section2.wav
44100
(108275654,)
c:\Users\michaelshih\Documents\Podcast_tmp\EP020\postedit\Section 3\Ep20_HWC_section3.wav
44100
(30277500, 2)
c:\Users\michaelshih\Documents\Podcast_tmp\EP020\postedit\Section 3\Ep20_Angel_section3.wav
44100
(30277500, 2)
c:\Users\michaelshih\Documents\Podcast_tmp\EP020\postedit\Section 3\Ep20_YY_section3.wav
44100
(30277500, 2)
c:\Users\michaelshih\Documents\Podcast_tmp\EP020\postedit\Section 4\Ep20_HWC_section4.wav
44100
(57283038, 2)
c:\Users\michaelshih\Documents\Podcast_tmp\EP020\postedit\Section 4\Ep20_Angel_section4.wav
44100
(57283038, 2)
c:\Users\michaelshih\Documents\Podcast_tmp\EP020\postedit\Section 4\Ep20_YY_section4.wav
44100
(57283038, 2)


In [59]:
def wav2mergemono(ipfilelist_dict, opmonofilelist, folderlist, namelistorder, jsonname = 'metadata.json'):
    
    with open(p_scripts.joinpath(jsonname), 'r') as f:
        mainmetadata_dict = json.load(f)

    maxlength_dict = {}
    for section in folderlist:    
        audiolength = []
        for wavfile, value in mainmetadata_dict['postedit'][section].items():
            audiolength.append(value['shape'][0])

        maxlength_dict[section] = np.amax(audiolength)
    
    mainmetadata_dict['maxlength'] = maxlength_dict
    
    print(maxlength_dict)
    
    #with open(p_scripts.joinpath(jsonname), 'w') as f:
    #     json.dump(mainmetadata_dict, f) 
    
    for name in namelistorder:
        print(name)
        if not opmonofilelist[name].is_file():
            wavtmp = []
            for section in folderlist:
                wavfile = ipfilelist_dict[name][section][0]
                fs, data = read(str(wavfile))
                
                if data.ndim == 2:
                    data = data[:, 0]
                    data = data.astype('int16')
                print(data.dtype)
                print(data.ndim)
                print(data.shape)
                
                data_tmp = np.pad(data, (0, mainmetadata_dict['maxlength'][section] - data.shape[0]), 'constant')
                wavtmp.append(data_tmp)
            wavtmp = np.concatenate(wavtmp, axis = 0)
            write(opmonofilelist[name], fs, wavtmp)
    
    return mainmetadata_dict
    
mainmetadata_dict = wav2mergemono(ipfilelist_dict, opmonofilelist, folderlist, namelistorder, jsonname = 'metadata.json')
print(mainmetadata_dict)

{'Section 1': 17518491, 'Section 2': 108275654, 'Section 3': 30277500, 'Section 4': 57283038}
HWC
int16
1
(17518491,)
int16
1
(108275654,)


C:\Users\michaelshih\AppData\Local\Temp\ipykernel_66340\969769860.py:27: WavFileWarning: Chunk (non-data) not understood, skipping it.
  fs, data = read(str(wavfile))


int16
1
(30277500,)
int16
1
(57283038,)
Angel
int16
1
(17518491,)
int16
1
(108275654,)
int16
1
(30277500,)
int16
1
(57283038,)
YY
int16
1
(17518491,)
int16
1
(108275654,)
int16
1
(30277500,)
int16
1
(57283038,)
{'postedit': {'Section 1': {'Ep20_HWC_section1.wav': {'fs': 44100, 'shape': [17518491]}, 'Ep20_Angel_section1.wav': {'fs': 44100, 'shape': [17518491]}, 'Ep20_YY_section1.wav': {'fs': 44100, 'shape': [17518491]}}, 'Section 2': {'Ep20_HWC_section2.wav': {'fs': 44100, 'shape': [108275654]}, 'Ep20_Angel_section2.wav': {'fs': 44100, 'shape': [108275654]}, 'Ep20_YY_section2.wav': {'fs': 44100, 'shape': [108275654]}}, 'Section 3': {'Ep20_HWC_section3.wav': {'fs': 44100, 'shape': [30277500, 2]}, 'Ep20_Angel_section3.wav': {'fs': 44100, 'shape': [30277500, 2]}, 'Ep20_YY_section3.wav': {'fs': 44100, 'shape': [30277500, 2]}}, 'Section 4': {'Ep20_HWC_section4.wav': {'fs': 44100, 'shape': [57283038, 2]}, 'Ep20_Angel_section4.wav': {'fs': 44100, 'shape': [57283038, 2]}, 'Ep20_YY_section4.wav'

In [48]:
def getinfo2json():
    send_command('GetInfo: Format="JSON" Type="Tracks"')
    response = get_response_json()
    result = json.loads(response)
    return(result)

trackinfo = getinfo2json()
print(trackinfo)

filename_noext = [re.sub('.wav', '', value.name) for idx, value in opmonofilelist.items()]

tracknamelist = [trackinfo['name'] for trackinfo in trackinfo]
name_tag = 'EP20'

def importfiles(filepath):
    do_command(f'Import2: Filename={filepath}')

TRACK_HEIGHT = 80
TRACK_OFFSET = 29.6
MATERIAL_DIR = Path('Z:\sc2.shih\Drive\Podcast\Edit\material')
OPMUSIC_PATH = MATERIAL_DIR.joinpath('direct_import', 'direct_import_op.wav')
ENDMUSIC_PATH = MATERIAL_DIR.joinpath('direct_import', 'direct_import_end.wav')
ENDCREDIT_PATH = MATERIAL_DIR.joinpath('direct_import', 'direct_import_endcredit.wav')
ENDCREDIT_OFFSET = 9.2

for idx, name in enumerate(namelistorder):
    track_name = f'{name_tag}_{name}'
    if track_name not in tracknamelist:
        print(f'import track: {track_name}')
        importfiles(p_mono.joinpath(f'{track_name}.wav'))
    do_command('SelectAll:')        
    do_command(f'SelectTracks: Mode="Set" Track="{idx}" TrackCouint="1"')
    do_command('ZoomSel')
    do_command(f'SetTrack: Height={str(TRACK_HEIGHT)}')
    do_command(f'SetClip: At="0" Start="{TRACK_OFFSET}"')

trackinfo = getinfo2json()
tracknamelist = [trackinfo['name'] for trackinfo in getinfo2json()]
tracklengthlist = [trackinfo['end'] - trackinfo['start'] for trackinfo in trackinfo]
TRACK_LENGTH_MAX = np.amax(tracklengthlist)

if 'opmusic' not in tracknamelist:
    do_command(f'Import2: Filename="{OPMUSIC_PATH}"')
    do_command(f'SelectTracks: Mode="Set" Track="{len(tracknamelist)}" TrackCouint="1"')
    do_command(f'SetTrack: Name="opmusic" Height={int(TRACK_HEIGHT*0.6)}')

trackinfo = getinfo2json()
tracknamelist = [trackinfo['name'] for trackinfo in getinfo2json()]

if 'endmusic' not in tracknamelist:
    do_command(f'Import2: Filename="{ENDMUSIC_PATH}"')
    do_command(f'SelectTracks: Mode="Set" Track="{len(tracknamelist)}" TrackCouint="1"')
    do_command(f'SetTrack: Name="endmusic" Height={int(TRACK_HEIGHT*0.6)}')
    do_command(f'SetClip: At="0" Start="{TRACK_OFFSET + TRACK_LENGTH_MAX}"')

trackinfo = getinfo2json()
tracknamelist = [trackinfo['name'] for trackinfo in getinfo2json()]

if 'endcredit' not in tracknamelist:
    do_command(f'Import2: Filename="{ENDCREDIT_PATH}"')
    do_command(f'SelectTracks: Mode="Set" Track="{len(tracknamelist)}" TrackCouint="1"')
    do_command(f'SetTrack: Name="endcredit" Height={int(TRACK_HEIGHT*0.6)}')
    do_command(f'SetClip: At="0" Start="{TRACK_OFFSET + TRACK_LENGTH_MAX + ENDCREDIT_OFFSET}"')



Send: >>> 
GetInfo: Format="JSON" Type="Tracks"
[]
import track: EP20_HWC
Send: >>> 
Import2: Filename=c:\Users\michaelshih\Documents\Podcast_tmp\EP020\postedit_mono\EP20_HWC.wav
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
SelectAll:
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
SelectTracks: Mode="Set" Track="0" TrackCouint="1"
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
ZoomSel
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
SetTrack: Height=80
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
SetClip: At="0" Start="29.6"
Rcvd: <<< 

BatchCommand finished: OK

import track: EP20_Angel
Send: >>> 
Import2: Filename=c:\Users\michaelshih\Documents\Podcast_tmp\EP020\postedit_mono\EP20_Angel.wav
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
SelectAll:
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
SelectTracks: Mode="Set" Track="1" TrackCouint="1"
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
ZoomSel
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
SetTrack: H

In [5]:
import pandas as pd
from datetime import datetime, timedelta
df = pd.read_csv(f'addmusic.csv')

for index, row in df.iterrows():
    hhmmss = row['timestamp']
    filename = row['music']
    MIDDLEPATH = MATERIAL_DIR.joinpath('02_transition', filename)

    [hours, minutes, seconds] = [int(x) for x in hhmmss.split(':')]
    MIDDLE_TIMESTAMP = timedelta(hours=hours, minutes=minutes, seconds=seconds).seconds
    print(MIDDLE_TIMESTAMP)

    trackinfo = getinfo2json()
    tracknamelist = [trackinfo['name'] for trackinfo in getinfo2json()]

    clipname = f'middle_{index + 1}'
    if clipname not in tracknamelist:
        do_command(f'Import2: Filename="{MIDDLEPATH}"')
        do_command(f'SelectTracks: Mode="Set" Track="{len(tracknamelist)}" TrackCouint="1"')
        do_command(f'SetTrack: Name="{clipname}" Height={int(TRACK_HEIGHT*0.6)}')
        do_command(f'SetClip: At="0" Start="{TRACK_OFFSET + MIDDLE_TIMESTAMP}"')
    

948
Send: >>> 
GetInfo: Format="JSON" Type="Tracks"
Send: >>> 
GetInfo: Format="JSON" Type="Tracks"
Send: >>> 
Import2: Filename="Z:\sc2.shih\Drive\Podcast\Edit\material\02_transition\Middle_01.wav"
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
SelectTracks: Mode="Set" Track="7" TrackCouint="1"
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
SetTrack: Name="middle_1" Height=48
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
SetClip: At="0" Start="977.6"
Rcvd: <<< 

BatchCommand finished: OK

1870
Send: >>> 
GetInfo: Format="JSON" Type="Tracks"
Send: >>> 
GetInfo: Format="JSON" Type="Tracks"
Send: >>> 
Import2: Filename="Z:\sc2.shih\Drive\Podcast\Edit\material\02_transition\Middle_01.wav"
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
SelectTracks: Mode="Set" Track="8" TrackCouint="1"
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
SetTrack: Name="middle_2" Height=48
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
SetClip: At="0" Start="1899.6"
Rcvd: <<< 

BatchCommand fi

In [7]:
# compressor

trackinfo = getinfo2json()
tracknamelist = [trackinfo['name'] for trackinfo in getinfo2json()]

for idx, name in enumerate(namelistorder):
    track_name = f'{name_tag}_{name}'
    do_command('SelectAll:')        
    do_command(f'SelectTracks: Mode="Set" Track="{idx}" TrackCouint="1"')
    do_command('ZoomSel')
    do_command(f'Compressor: Threshold={str(-12)} NoiseFloor={str(-35)} Ratio={str(2)}')


Send: >>> 
GetInfo: Format="JSON" Type="Tracks"
Send: >>> 
GetInfo: Format="JSON" Type="Tracks"
Send: >>> 
SelectAll:
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
SelectTracks: Mode="Set" Track="0" TrackCouint="1"
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
ZoomSel
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
Compressor: Threshold=-12 NoiseFloor=-35 Ratio=2
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
SelectAll:
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
SelectTracks: Mode="Set" Track="1" TrackCouint="1"
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
ZoomSel
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
Compressor: Threshold=-12 NoiseFloor=-35 Ratio=2
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
SelectAll:
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
SelectTracks: Mode="Set" Track="2" TrackCouint="1"
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
ZoomSel
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
Compressor: Threshold=-12 NoiseFloo